# Counterparty Credit Risk & Backtesting MVP

 Private Credit | Stress Testing | Pre-Trade What-If

Live demo: https://and86rey.github.io/ccr-backtest-mvp

In [ ]:
import pandas as pd
from core.exposure import CCRSimulator
import matplotlib.pyplot as plt
%matplotlib inline

trades = pd.read_csv('../data/sample_portfolio.csv', parse_dates=['maturity_date'])
sim = CCRSimulator(n_paths=2500)
profiles = sim.run_netting_set(trades)

## 1. Current Exposure Profile (NS001 – Deutsche Bank)

In [ ]:
df = profiles['NS001']
plt.figure(figsize=(12,6))
plt.plot(df['time'], df['EE_total']/1e6, label='EE', linewidth=3)
plt.plot(df['time'], df['PFE_total']/1e6, label='PFE 95%', linewidth=3, color='red')
plt.title('NS001 – Deutsche Bank (Baseline)', fontsize=16)
plt.xlabel('Years'); plt.ylabel('€ millions')
plt.legend(); plt.grid(alpha=0.3); plt.show()

## 2. Limits & EWI Dashboard

In [ ]:
from analytics.limits_ewi import apply_limits_and_ewi
from IPython.display import display, HTML

dashboard = apply_limits_and_ewi(profiles, trades)
styled = dashboard.style \
    .apply(lambda x: ['background: #ff4444; color: white;' if v=='BREACH' else '' for v in x], subset=['Status']) \
    .set_table_attributes('class="styled-table"')

display(HTML(f'<div class="styled-table">{styled.to_html()}</div>'))

## 3. Stress Testing & Wrong-Way Risk (Private Equity example)

In [ ]:
from analytics.stress import stress_exposure, wrong_way_risk_uplift

ns003 = profiles['NS003']
stressed = stress_exposure(ns003, shock_factor=1.6, vol_multiplier=2.5)
wwr = wrong_way_risk_uplift(stressed, 'Private Equity')

plt.figure(figsize=(12,6))
plt.plot(ns003['time'], ns003['PFE_total']/1e6, label='Baseline PFE', linewidth=3)
plt.plot(wwr['time'], wwr['PFE_total']/1e6, label='Stressed + WWR (PE)', linewidth=3, color='darkred')
plt.title('NS003 – PE Fund Gamma: Stress + Wrong-Way Risk Uplift')
plt.legend(); plt.grid(); plt.show()

## 4. Pre-Trade What-If – Adding a €5m distressed credit TRS

In [ ]:
from analytics.pretrade import what_if_new_trade

new_trade = {
    'netting_set_id': 'NS003',
    'counterparty': 'Distressed Credit Fund',
    'product': 'Total Return Swap',
    'notional': 5_000_000,
    'maturity_date': '2028-12-31',
    'strike_or_rate': 0.08,
    'rating': 'B',
    'sector': 'Private Equity'
}

impact = what_if_new_trade(profiles, new_trade, trades)
pd.DataFrame([impact])

## 5. Backtesting Result (2018–2024)

In [ ]:
from analytics.backtest import run_historical_backtest
from IPython.display import display, HTML

bt = run_historical_backtest()
bt_df = pd.DataFrame([bt])
styled_bt = bt_df.style \
    .apply(lambda x: ['background: #44ff44; color: black;' if v else 'background: #ff4444; color: white;' for v in x], subset=['Pass']) \
    .set_table_attributes('class="styled-table"')

display(HTML(f'<div class="styled-table">{styled_bt.to_html()}</div>'))